# Определение стоимости автомобилей

**Признаки:**
* DateCrawled — дата скачивания анкеты из базы
* VehicleType — тип автомобильного кузова
* RegistrationYear — год регистрации автомобиля
* Gearbox — тип коробки передач
* Power — мощность (л. с.)
* Model — модель автомобиля
* Kilometer — пробег (км)
* RegistrationMonth — месяц регистрации автомобиля
* FuelType — тип топлива
* Brand — марка автомобиля
* NotRepaired — была машина в ремонте или нет
* DateCreated — дата создания анкеты
* NumberOfPictures — количество фотографий автомобиля
* PostalCode — почтовый индекс владельца анкеты (пользователя)
* LastSeen — дата последней активности пользователя

**Целевой признак:**
* Price — цена (евро)

**Для оценки качества моделей применяется метрика RMSE.**

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
from catboost import CatBoostRegressor
from sklearn.preprocessing import OrdinalEncoder
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression



In [2]:
df = pd.read_csv('/datasets/autos.csv')

In [3]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
df.duplicated().sum()

4

In [6]:
df = df.drop_duplicates()
display(df.head())

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [7]:
df.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

In [8]:
# так как эти параметры имеюют важное значения для ценообразования, поэтому пропуски с ними удаляем
df.dropna(subset=['Model', 'VehicleType', 'Gearbox', 'FuelType'], inplace=True)

In [9]:
# данные не имеют отношения к техническим характеристикам, поэтому не влияют на стоимось. Необходимости в них нет
df.drop(['DateCrawled', 'RegistrationMonth', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'], axis=1, inplace=True)

In [10]:
# пропуски в столбце с большой долей вероятности могут означать, что машина не была в ремонте, поэтому заполняю соответствующе
df['NotRepaired'] = df['NotRepaired'].fillna('No')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284122 entries, 2 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             284122 non-null  int64 
 1   VehicleType       284122 non-null  object
 2   RegistrationYear  284122 non-null  int64 
 3   Gearbox           284122 non-null  object
 4   Power             284122 non-null  int64 
 5   Model             284122 non-null  object
 6   Kilometer         284122 non-null  int64 
 7   FuelType          284122 non-null  object
 8   Brand             284122 non-null  object
 9   NotRepaired       284122 non-null  object
dtypes: int64(4), object(6)
memory usage: 23.8+ MB


In [12]:
df.corr()

,Price,RegistrationYear,Power,Kilometer
Price,1.000000,0.548496,0.167917,-0.386358
RegistrationYear,0.548496,1.000000,0.063803,-0.325723
Power,0.167917,0.063803,1.000000,0.022887
Kilometer,-0.386358,-0.325723,0.022887,1.000000


In [13]:
df.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,No
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,petrol,bmw,yes
6,2200,convertible,2004,manual,109,2_reihe,150000,petrol,peugeot,no


In [14]:
df.sort_values(by=['Power']).head(25)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
187514,1200,small,2003,manual,0,clio,150000,petrol,renault,no
57562,1700,wagon,1998,manual,0,passat,150000,petrol,volkswagen,no
57558,2950,small,2007,manual,0,note,100000,petrol,nissan,yes
57547,700,sedan,1999,manual,0,octavia,150000,petrol,skoda,no
166074,1800,sedan,1993,manual,0,3er,150000,petrol,bmw,No
57535,2600,bus,2005,manual,0,golf,150000,gasoline,volkswagen,yes
166082,700,small,1999,manual,0,2_reihe,150000,petrol,peugeot,no
303459,1450,wagon,1998,auto,0,vectra,150000,petrol,opel,no
303460,2900,bus,2006,manual,0,modus,80000,petrol,renault,no
57521,1100,small,1998,manual,0,corsa,150000,petrol,opel,no


In [15]:
drop_index = df[(df['Power'] <= 5.0)].index

In [16]:
#удаляю строки, где мощность автомобиля меньше или равна 5 л.с.
df = df.drop(drop_index, axis=0)

In [17]:
cat_features = ['VehicleType', 'RegistrationYear', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
for feature in cat_features:
    df[feature] = df[feature].astype('category')

In [19]:
features = df.drop(['Price'], axis=1)

In [20]:
target = df['Price']

In [21]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.4,
                                                                              random_state=12345)

In [22]:
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.5, 
                                                                            random_state=12345)

In [23]:
print(features_valid.shape, features_test.shape, features_train.shape)

(53603, 9) (53603, 9) (160807, 9)


**Выводы:**
* данные загружены 
* выявлены пропуски в столбцах с техническими характеристиками. Так как заполнить их не представляется возможным, и к тому же  объем данных позволяет, поэтому принято решение их удалить.
* пропуски в столбце с информацией о ремонте заполнены отрицательно, основываясь на предположении, что данные об отсутствие ремонта просто не вносили в анкету. 
* удалены выбросы по мощности
* есть небольшая корреляция между ценой и датой регистрации авто.
* котегориальные признаки выделены в отдельный список.
* данные разделены на обучающиеб валидационные и тестовые в пропорции 60:20:20.

## Обучение моделей

### CatBoost

In [24]:
#cat_features = ['VehicleType', 'RegistrationYear', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
cat_model = CatBoostRegressor(loss_function='RMSE', iterations=300, depth=15)

In [25]:
%%time
cat_model.fit(features_train, target_train, cat_features=cat_features, verbose=False)

CPU times: user 7min 49s, sys: 29.6 s, total: 8min 19s
Wall time: 8min 19s


In [26]:
%%time
predicted_cat = cat_model.predict(features_valid)
result_cb = mean_squared_error(target_valid, predicted_cat) ** 0.5

CPU times: user 671 ms, sys: 211 µs, total: 671 ms
Wall time: 678 ms


In [27]:
print('RMSE cat_model:', result_cb)

RMSE cat_model: 1617.5880026512905


Данная модель показала хороший результат, и времени для обучения требуется немного.

### LigthGBM

In [28]:
%%time
from sklearn.model_selection import GridSearchCV

score_func = make_scorer(mean_squared_error, greater_is_better=False)

model_lg = LGBMRegressor()#n_estimators=500, max_depth=30, random_state=12345)
params = {'n_estimators': range(50, 500, 50), 'max_depth': range(20, 60,10)}
gridSearchCV = GridSearchCV(estimator = model_lg, 
    param_grid = params, 
    scoring=score_func)
gridSearchCV.fit(features_train,target_train)
predicted_lg = gridSearchCV.predict(features_valid)

result_lg = (mean_squared_error(target_valid, predicted_lg) ** 0.5)

print('RMSE LGBMRegressor:', result_lg)

RMSE LGBMRegressor: 1607.677012734649
CPU times: user 23min 40s, sys: 8.77 s, total: 23min 49s
Wall time: 24min 16s


In [29]:
gridSearchCV.best_params_


{'max_depth': 20, 'n_estimators': 450}

Результат этой модели лучше, но зато времени требуется в несколько раз больше

### Random Forest

In [30]:
#кодирую категориальные признаки и объединяю с оставшимися
train_ohe = pd.get_dummies(features_train[cat_features], drop_first=True)
features_train_ohe = pd.concat([train_ohe, features_train['Power'], features_train['Kilometer']], axis=1)

valid_ohe = pd.get_dummies(features_valid[cat_features], drop_first=True)
features_valid_ohe = pd.concat([valid_ohe, features_valid['Power'], features_valid['Kilometer']], axis=1)

test_ohe = pd.get_dummies(features_test[cat_features], drop_first=True)
features_test_ohe = pd.concat([test_ohe, features_test['Power'], features_test['Kilometer']], axis=1)

In [31]:
%%time
model_rf = RandomForestRegressor(n_estimators = 300, max_depth=15, random_state=12345)
#grid_SCV = GridSearchCV(estimator = model_rf, 
#    param_grid = params, 
#    scoring=score_func)   
model_rf.fit(features_train_ohe, target_train)
predicted_rf = model_rf.predict(features_valid_ohe)
result_rf = (mean_squared_error(target_valid, predicted_rf) ** 0.5)

print('RMSE RandomForestRegressor:', result_rf)

RMSE RandomForestRegressor: 1947.5373248915218
CPU times: user 14min 53s, sys: 798 ms, total: 14min 54s
Wall time: 14min 54s


Результат гораздо хуже, и по времени тоже

## Анализ моделей

In [32]:
predict = cat_model.predict(features_test)
best_result = mean_squared_error(target_test, predict) ** 0.5
print('RMSE: {:2f}'.format(best_result))


RMSE: 1634.161100


In [34]:
# результаты обучения разных моделей
r = pd.DataFrame([['catboost', result_cb, '8min 19s'],
                 ['LigthGBM', result_lg, '24min 16s'],
                 ['random_forest', result_rf, '14min 54s'],
                 ], 
                columns=['model', 'RMSE', 'time to learn'])
display(r)

,model,RMSE,time to learn
0,catboost,1617.588003,8min 19s
1,LigthGBM,1607.677013,24min 16s
2,random_forest,1947.537325,14min 54s


Самый оптимальный результат, на мой взгляд, у модели CatBoost - лучшее соотношение RMSE и времени для обучения.

## Выводы:
* данные загружены
* удалены пропуски в столбцах с техническими характеристиками. 
* пропуски в столбце с информацией о ремонте заполнены.
* применено порядковое кодирование категориальных признаков.
* данные разделены на обучающие, валидационные и тестовые в пропорции 60:20:20.
* обучены несколько моделей.
* проанализированы результаты и выбрана лучшая модель - LigthGBM с лучшим соотношение качества и времени.